In [1]:
import requests
import datetime
import threading
import pandas as pd

In [2]:
'''
import custom module 'access.py'
to access your own API token and log in

$ cat access.py 
MY_KEY = "Bearer xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

'''

import access
API_KEY = access.MY_KEY

In [3]:
'''
set empty python dictionary to store data
'''

data = {'ip': [],
       'location': [],
       'connected_time': []}

In [4]:
def snapshot():
    
    headers = {'Authorization': API_KEY}   
    res = requests.get('https://streaming.lahmacun.hu/api/station/1/listeners', headers=headers)
    results = res.json()
    # print('current listeners:', len(results))
    
    for listener in results:
        
        connected_time = listener['connected_time']
        connected_time = datetime.timedelta(seconds=connected_time)
        ip = listener['ip']
        if listener['location']['status'] == 'error':
            loc = 'N/A'
        else:
            loc = listener['location']['country']
        
        if ip in data['ip']:
            i = data['ip'].index(ip)
            if connected_time >= data['connected_time'][i][-1][1]:
                data['connected_time'][i][-1][1] = connected_time
            else:
                data['connected_time'][i].append([datetime.datetime.now(), connected_time])                
        else: 
            data['ip'].append(ip)
            data['location'].append(loc)
            data['connected_time'].append([[datetime.datetime.now(), connected_time]])

In [9]:
'''
monitoring/debugging
'''

snapshot()
display(data)

{'ip': ['192.168.176.6'],
 'location': ['N/A'],
 'connected_time': [[[datetime.datetime(2021, 11, 1, 15, 9, 42, 899628),
    datetime.timedelta(seconds=34615)],
   [datetime.datetime(2021, 11, 1, 15, 9, 42, 899643),
    datetime.timedelta(seconds=34645)],
   [datetime.datetime(2021, 11, 1, 15, 10, 12, 904031),
    datetime.timedelta(seconds=34675)],
   [datetime.datetime(2021, 11, 1, 15, 10, 42, 945853),
    datetime.timedelta(seconds=34705)],
   [datetime.datetime(2021, 11, 1, 15, 11, 12, 916061),
    datetime.timedelta(seconds=34735)],
   [datetime.datetime(2021, 11, 1, 15, 11, 42, 917705),
    datetime.timedelta(seconds=34759)],
   [datetime.datetime(2021, 11, 1, 15, 12, 6, 384232),
    datetime.timedelta(seconds=2389)]]]}

In [6]:
'''
-> populates/updates 'data' dictionary every 30 seconds
'''

def autoFetch():
    threading.Timer(30.0, autoFetch).start()
    snapshot()
    # df = pd.DataFrame(data)
    # display(df)

In [7]:
'''
-> exports a snapshot of 'data' dictionary every n hours
'''

def autoExport(n = 1):
    due = n * 3600.0
    threading.Timer(due, autoExport).start()
    timestamp = datetime.datetime.now()
    df = pd.DataFrame(data)
    df.to_csv('lahmacun_' + str(timestamp) + '.csv')

In [8]:
autoFetch()
autoExport(4) # exports every 4 hours